# 수치/기초/기계 고장 여부 및 고장 범주 분류
- 이 노트북은 셀을 차례로 실행하여 이미지 과제의 전반적인 과정을 수행해볼 수 있게 제작되었습니다.

## 과제 설명
- 기계 고장 예지 센서 데이터를 통해 각 기계가 정상인지 비정상인지, 만약 비정상이라면 어떠한 고장의 범주에 속하는지를 예측하는 과제

## 데이터 설명
- 입력 데이터 feature
  - x, y, z : 3상 전류의 각 상
- 출력 데이터 label
  - 단위 시간당 수집된 센서 데이터의 오류 여부 및 오류의 종류이며 그 종류는 “정상”, “베어링 불량”, “회전체 불평형”, “축정렬 불량”, “벨트 느슨함”
- 데이터셋 구성
  - train: t(time), x, y, z 등의 feature 정보가 담긴 10000개의 csv 파일과 label 정보가 담긴 1개의 csv 파일(train.csv)
  - test: t(time), x, y, z 등의 feature 정보가 담긴 2000개의 csv 파일
  
## 자주 사용되는 RNN 모델
- LSTM, GRU 등

## 코드 구조
이 베이스라인 코드는 간단하게 아래 네 단계로 이루어져 있습니다.
- `1.데이터`: 사용할 데이터셋을 가져오고 모델에 전달할 Dataloader 생성
  - `class CustomDataset`: 데이터를 불러오고 (필요할 경우) 데이터 전처리 진행, 및 torch.utils.data.DataLoader의 첫번째 인자 형식으로 변환
  - `torch.utils.data.DataLoader(dataset, batch_size=, ...)`: 모델에 공급할 데이터 로더 생성
- `2.모델 설계`: 학습 및 추론에 사용할 모델 구조 설계
  - `class LSTMClassifier`: 모델 구조 설계
- `3.학습`: 설계된 모델로 데이터 학습
  - 학습된 모델은 `.ipynb` 코드와 같은 경로에 저장됨
- `4.추론`: 학습된 모델을 사용해 테스트 데이터로 추론
  - 학습된 모델로 테스트 데이터에 대한 추론을 진행
  - 추론 결과는 `.ipynb` 코드와 같은 경로에 저장됨. 이를 플랫폼에 업로드해 점수 확인

## 세팅
### 라이브러리
- 코드 전반에 사용되는 라이브러리를 설치 및 로드합니다.

In [6]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
# !pip install torch
# !pip install 

In [11]:
# 필요한 라이브러리 불러오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F

import os
import sys

In [7]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
if device == torch.device('cuda'):
    print( torch.cuda.get_device_properties( device ))
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# 경로 설정
DATA_DIR = '/workspace/01_data/18_machine/02_processed/d3'

cuda
_CudaDeviceProperties(name='Tesla V100-PCIE-32GB', major=7, minor=0, total_memory=32480MB, multi_processor_count=80)


## 데이터 및 전처리
- file name과 label 정보가 담긴 meta data(train.csv)를 통해 개별 기계시설물 csv에 맵핑하여 데이터 로드 작업 수행
- 10000개의 train dataset을 train / val set(8:2의 비율)으로 분할 
- 총 5개 범주에 대한 인코딩

In [14]:
# validation set ratio
VAL_RATIO = 0.2

class CustomDataset(Dataset):
    def __init__(self, data_dir, data_meta, mode, sequence_length=2000):
        """
        data_meta : train.csv, test.csv etc. This is the file that contains file_name, label, etc.
        
        """
        
        self.data_dir = os.path.join(data_dir, 'train')
        self.data_meta = os.path.join(data_dir, data_meta)
        self.mode = mode
        self.seq_len = sequence_length # 데이터 파일 하나당 행 갯수
        self.label_onehot = {
            '축정렬불량': 0,
            '회전체불평형': 1,
            '베어링불량': 2,
            '벨트느슨함': 3,
            '정상': 4,
        }
        self.db, self.label = self.data_loader()

        
    def data_loader(self):
        
        # 파일 없을때 뜨는 메세지
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        if not os.path.lexists(self.data_meta):
            print(f'!!! Cannot find {self.data_meta}... !!!')
            sys.exit()
            
        # 일단 클래스별로 나열돼있음.
        meta = pd.read_csv(self.data_meta)
        
        # 학습 데이터 클래스 상관 없이 섞어버리기
        meta = meta.sample(frac=1).reset_index(drop=True) 

        # db_train, db_val 선언
        db_train = db_val= pd.DataFrame(columns=['file_name','label'])
 
        for la in self.label_onehot.keys():
            df = meta[meta['label']==la]
            
            # 학습, 검증 데이터 분할
            train, val = train_test_split(df, test_size=VAL_RATIO, random_state=42, shuffle=True)
            
            db_train = pd.concat([db_train,train])
            db_val = pd.concat([db_val,val])
        
        db_train = db_train.sample(frac=1).reset_index(drop=True)
        db_val = db_val.sample(frac=1).reset_index(drop=True)

       
        if self.mode=='train':
            db = db_train
            
            # label 숫자값으로 바꾸기
            label = pd.DataFrame(db_train['label'].map(self.label_onehot), columns=['label'])
            
        elif self.mode=='val':
            db = db_val
            
            # label 숫자값으로 바꾸기
            label = pd.DataFrame(db_val['label'].map(self.label_onehot), columns=['label'])
        else:
            print("Please check your mode : ", mode, " must be either train or val")
        

        # db returns something like train.csv (filename, label(str), rms, ....)
        # label returns label(integer) dataframe       
        return db, label
        

    def __len__(self):
        
        return len(self.label)
    
    # idx(파일 넘버)에 관한 정보 추출
    def __getitem__(self, idx):
        filename = self.db.iloc[idx]['file_name']
        label_folder = self.db.iloc[idx]['label']
        
        file_path = os.path.join(self.data_dir, label_folder, filename)
        data = pd.read_csv(file_path)
        label = self.label.iloc[idx]['label']
        # t는 시간이므로 feature가 아니라 제거
        data = data.drop(columns=['t']).values.astype('float32')
        
        return_dict = {'data': data, 'label': label}
        return return_dict

In [15]:
trainset = CustomDataset(DATA_DIR, "train.csv", 'train')
valset = CustomDataset(DATA_DIR, "train.csv", 'val')

batch_size=64
trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
valloader = DataLoader(valset, shuffle=False, batch_size=batch_size)

Loading train dataset..
Loading val dataset..


## 모델 설계
### 사용할 파라미터
- `LEARNING_RATE` : 경사하강법(Gradient Descent)을 통해 loss function의 minimum값을 찾아다닐 때, 그 탐색 과정에 있어서의 보폭 정도로 직관적으로 이해 할 수 있습니다. 보폭이 너무 크다면 최적값을 쉽게 지나칠 위험이 있고, 보폭이 너무 작다면 탐색에 걸리는 시간이 길어집니다.
- `EPOCHS` : 
  - 한 번의 epoch는 인공 신경망에서 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것입니다.
  - 즉, epoch이 1만큼 지나면, 전체 데이터 셋에 대해 한번의 학습이 완료된 상태입니다.
  - 모델을 만들 때 적절한 epoch 값을 설정해야만 underfitting과 overfitting을 방지할 수 있습니다.
  - 1 epoch = (데이터 갯수 / batch size) interations

In [17]:
# hyper-parameters
lr = 0.001
n_epochs = 10
iterations_per_epoch = len(trainloader)
best_acc = 0
patience, patience_counter = 50, 0

In [18]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=256, num_layers=2, output_dim=5, dropout=0):
        """
        input_dim = number of features at each time step 
                    (number of features given to each LSTM cell)
        hidden_dim = number of features produced by each LSTM cell (in each layer)
        num_layers = number of LSTM layers
        output_dim = number of classes of motor anomaly 
        """
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, 
                            num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
    def forward(self, X):
        hidden_features, (h_n, c_n) = self.lstm(X)  # (h_0, c_0) default to zeros
        hidden_features = hidden_features[:,-1,:]  # index only the features produced by the last LSTM cell
        out = self.fc(hidden_features)
        return out

In [19]:
model = LSTMClassifier(dropout=0.75)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## 학습

In [20]:
print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    # initialize losses
    loss_train_total = 0
    loss_val_total = 0
    
    # Training loop
    for i, batch_data in enumerate(trainloader):
        model.train()
        X_batch = batch_data['data'].to(device)
        y_batch = batch_data['label'].to(device).long()
        optimizer.zero_grad()
        
        y_pred = model(X_batch) 
        loss = criterion(y_pred, y_batch)
        loss_train_total += loss.cpu().detach().item() * batch_size
        
        loss.backward()
        optimizer.step()
    
    loss_train_total = loss_train_total / len(trainset)
    
    
    # Validation loop
    with torch.no_grad():
        for i, batch_data in enumerate(valloader):
            model.eval()
            X_batch = batch_data['data'].to(device)
            y_batch = batch_data['label'].to(device).long()

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss_val_total += loss.cpu().detach().item() * batch_size

    loss_val_total = loss_val_total / len(valset)
    
    
    # Validation Accuracy
    correct, total = 0, 0
    with torch.no_grad():
        model.eval()
        for i, batch_data in enumerate(valloader):
            X_batch = batch_data['data'].to(device)
            y_batch = batch_data['label'].to(device).long()

            y_pred = model(X_batch)
            class_predictions = F.log_softmax(y_pred, dim=1).argmax(dim=1)
            total += y_batch.size(0)
            correct += (class_predictions == y_batch).sum().item()

    acc = correct / total

    
    # Logging
    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Train Loss: {loss_train_total:.4f}. Val Loss: {loss_val_total:.4f} Acc.: {acc:2.2%}')

    if acc > best_acc:
        patience_counter = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best_211101.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 83.85%
Epoch:   5. Train Loss: 0.9970. Val Loss: 1.3440 Acc.: 41.80%
Epoch 7 best model saved with accuracy: 86.55%
Epoch:  10. Train Loss: 1.4494. Val Loss: 1.4867 Acc.: 29.20%


# 추론

In [33]:
DATA_DIR = '/workspace/01_data/18_machine/02_processed/d3/'

class TestDataset(Dataset):
    def __init__(self, data_dir, sequence_length=2000):
        """
        data_meta : train.csv. This is the file that contains file_name, label, etc.
        
        """
        
        self.data_dir = os.path.join(data_dir, 'test')
        self.seq_len = sequence_length # 데이터 파일 하나당 행 갯수
        self.db = self.test_loader()


    def test_loader(self):
       
        # 파일 없을때 뜨는 메세지
        print('Loading ' + 'test' + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()

        db_test = os.listdir(self.data_dir)
        if '.ipynb_checkpoints' in db_test:
            db_test.remove('.ipynb_checkpoints')
        db_test = pd.DataFrame(db_test, columns=['file_name'])


        # db returns something like train.csv (filename, label(str), rms, ....)
        # label returns label(integer) dataframe       
        return db_test
  
    def __len__(self):
        
        return len(self.db)

    
    # idx(파일 넘버)에 관해 정보 뽑아냄
    def __getitem__(self, idx):
        filename = self.db.iloc[idx]['file_name']
        file_path = os.path.join(self.data_dir, filename)
        data = pd.read_csv(file_path)
        # t 는 시간이므로 feature가 아니라 제거
        data = data.drop(columns=['t']).values.astype('float32')
        
        return_dict = {'file_name':filename,'data': data}
        return return_dict

In [34]:
testset = TestDataset(DATA_DIR, 'test')

batch_size= 64
testloader = DataLoader(testset, shuffle=False, batch_size=batch_size)

Loading test dataset..


In [35]:
model = LSTMClassifier(dropout=0.75)
model.to(device)
model.load_state_dict(torch.load('best_211101.pth'))

<All keys matched successfully>

In [36]:
pred_list = []
filename_list = []
with torch.no_grad():
    print('prediction start')
    model.eval()
    for i, batch_data in enumerate(testloader):
        X_batch = batch_data['data'].to(device)

        y_pred = model(X_batch)
        class_predictions = F.log_softmax(y_pred, dim=1).argmax(dim=1).tolist()
        pred_list.extend(class_predictions)
        filename_list.extend(batch_data['file_name'])
print("prediction finished")

prediction start
prediction finished


In [37]:
# 추론 결과디코딩
label_dict = {
            '축정렬불량': 0,
            '회전체불평형': 1,
            '베어링불량': 2,
            '벨트느슨함': 3,
            '정상': 4,
        }
decode = dict(map(reversed, label_dict.items()))
pred_list = list(map(lambda x: decode[x], pred_list))

In [38]:
df = pd.DataFrame({'file_name':filename_list, 'label':pred_list})

In [39]:
df.head()

,file_name,label
0,0.csv,벨트느슨함
1,1.csv,벨트느슨함
2,2.csv,축정렬불량
3,3.csv,벨트느슨함
4,4.csv,회전체불평형


In [40]:
# 제출 파일 제작
df.to_csv("/workspace/01_data/18_machine/02_processed/d3/prediction_211101.csv", index=False)